In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import heapq
from collections import defaultdict, Counter
import struct
import sys
from scipy.fftpack import dct, idct
from scipy.stats import entropy

# Funções úteis

In [11]:
def calcEntropy(image):
    hist = cv2.calcHist([image],[0],None,[256],[0,256])
    hist = hist.ravel()/hist.sum()
    logs = np.log2(hist+0.00001)
    entropy = -1 * (hist*logs).sum()
    return entropy

def divideImageIntoSubImages(image, subImageSize):
    subImages = []
    for i in range(0, image.shape[0], subImageSize):
        for j in range(0, image.shape[1], subImageSize):
            subImages.append(image[i:i+subImageSize, j:j+subImageSize])
    return np.array(subImages)

def reconstructImage(subimages, originalImageShape):
    image = np.zeros(originalImageShape)
    idx = 0
    for i in range(0, originalImageShape[0], subimages[0].shape[0]):
        for j in range(0, originalImageShape[1], subimages[0].shape[1]):
            image[i:i+subimages[0].shape[0], j:j+subimages[0].shape[1]] = subimages[idx]
            idx += 1
    return image

def getNumberOfDifferentColors(img):
    return len(np.unique(img))

def plotImage(img, title):
    plt.imshow(img, cmap='gray')
    plt.title(title)
    plt.show()

In [ ]:
ORIGINAL_IMAGE = cv2.imread('./imagens/unequal.pgm',0)
ORIGINAL_IMAGE_SHAPE = ORIGINAL_IMAGE.shape
plotImage(ORIGINAL_IMAGE, 'Imagem Original')

In [13]:
SUBIMAGES = divideImageIntoSubImages(ORIGINAL_IMAGE, 8)

# Tirar os residuos/  Pegar os resíduos e transformar na imagem original - Giovana 

In [ ]:
    
def generate_macroblocks(img, block_size):
    """
    Generates macroblocks of a given size from the image.
    """
    for y in range(0, img.shape[0], block_size):
        for x in range(0, img.shape[1], block_size):
            macroblock = img[y:y+block_size, x:x+block_size]
            yield y, x, macroblock

def get_vertical_prediction(macroblock,left_column = None):
    """
    Applies a predictive function without loss on each macro block based on
    the formula f(x, y) = f(x, y) - f(x, y - 1).
    """
    new_block = np.zeros_like(macroblock, dtype=float)
    if left_column is not None:
        new_block[:,:] = left_column
    else: 
        new_block[:,0] = 0
        new_block[:,1:] = macroblock[:,0].reshape(-1, 1) 
    # print(new_block,left_column)
    # new_block[:,:] = macroblock[:,0:]
    return new_block

def get_horizontal_prediction(macroblock, top_row=None):
    """
    Applies a vertical predictive function on a given macroblock based on the formula
    f(x, y) = f(x, y) - f(x-1, y). 
    """
    new_block = np.zeros_like(macroblock, dtype=float)
    if top_row is not None:
        new_block[:,:] = top_row.reshape(-1, 1) 
    else: 
        new_block[0,:] = 0
        new_block[1:,:] = macroblock[0,:].reshape(1, -1) 
    
    # new_block[1:,:] = macroblock[0:-1,:]
    
    return new_block

def get_mean_prediction(macroblock,left_column,top_row):

    block_size = macroblock.shape[0]
    new_block = np.zeros_like(macroblock, dtype=float)

    if left_column is not None and top_row is not None:
        mean_value = int((np.sum(top_row) + np.sum(left_column)) / (2 * block_size))
        new_block = np.full((block_size, block_size), mean_value)

    return new_block

def calculate_entropy(block):
    """
    Calculates the entropy of the given block.
    """
    # Flatten the block and calculate the probability distribution
    flattened_block = block.flatten()
    value_counts = Counter(flattened_block)
    
    # Calculate the probabilities of each unique value
    total_count = sum(value_counts.values())
    probabilities = np.array([count / total_count for count in value_counts.values()])
    
    # Calculate entropy
    return entropy(probabilities, base=2)

def get_best_prediction(original_block,block_vertical, block_horizontal,block_mean):
    """
    Compares two blocks by calculating their entropy and returns the block
    with the lowest entropy along with an indicator of which block was chosen.
    """
    # Calculate entropy for both blocks
    entropy_vertical = calculate_entropy(block_vertical)
    entropy_horizontal = calculate_entropy(block_horizontal)
    entropy_mean = calculate_entropy(block_mean)
    
    # Find the block with the minimum entropy
    entropies = [entropy_vertical, entropy_horizontal, entropy_mean]
    blocks = [block_vertical, block_horizontal, block_mean]
    min_entropy_index = entropies.index(min(entropies))
    
    # Return the block with the lowest entropy and its index
    return blocks[min_entropy_index],min_entropy_index
    
def process_image(img, block_size):
    """
    Processes the entire image by dividing it into macroblocks,
    applying the predictive function to each block, and 
    reconstructing the output image.
    """
    options = []  # To store the choice of predictive function (0 or 1)
    pred_img = np.zeros_like(img, dtype=float)
    
    # Iterate over each macroblock in the image
    for y, x, macroblock in generate_macroblocks(img, block_size):
        # Get left column and top row from the last macroblock for prediction
        left_column = img[y:y+block_size,x-1] if (x > 0) else None
        top_row = img[y-1, x:x+block_size] if (y > 0) else None

        block_vertical   = macroblock - get_vertical_prediction(macroblock,left_column) 
        block_horizontal = macroblock - get_horizontal_prediction(macroblock,top_row)
        block_mean = macroblock - get_mean_prediction(macroblock,left_column,top_row)



        # Select the best prediction based on entropy
        new_block, option = get_best_prediction(macroblock,block_vertical, block_horizontal,block_mean) ## testar remover e calcular a entropia
        options.append(option)

        # Place the processed block back into the image
        pred_img[y:y+block_size, x:x+block_size] = new_block

    return pred_img, options

def reconstruct_image(residual_img, block_size, pred_type):
    """
    Reconstructs the original image from a residual image.
    """
    # Create an empty array to store the reconstructed original image
    original_img = np.zeros_like(residual_img, dtype=float)

    i = 0
    # Iterate over the image in blocks of size `block_size`
    for y, x, residual_block in generate_macroblocks(residual_img, block_size):
        left_column = original_img[y:y+block_size,x-1] if (x > 0) else None
        top_row = original_img[y-1, x:x+block_size] if (y > 0) else None

        reconstructed_block = np.zeros_like(residual_block, dtype=float)
        
        # Extract the current residual block
        if pred_type[i] == 0:
            reconstructed_block = get_vertical_prediction(residual_block, left_column)
            
        
        elif pred_type[i] == 1:
            reconstructed_block = get_horizontal_prediction(residual_block, top_row)

        elif pred_type[i] == 2:
            reconstructed_block = get_mean_prediction(residual_block,left_column, top_row)
        
        # Place the reconstructed block back into the image
        original_img[y:y+block_size, x:x+block_size] = reconstructed_block + residual_block

        i+=1

    return original_img

    

In [ ]:
quantization_matrix = np.matrix([[16, 11, 10, 16, 24, 40, 51, 61], 
                [12, 12, 14, 19, 26, 58, 60, 55],
                [14, 13, 16, 24, 40, 57, 69, 56],
                [14, 17, 22, 29, 51, 87, 80, 62],
                [18, 22, 37, 56, 68, 109, 103, 77],
                [24, 35, 55, 64, 81, 104, 113, 92],
                [49, 64, 78, 87, 103, 121, 120, 101],
                [72, 92, 95, 98, 112, 100, 103, 99]]) /10

def get_dct(img):
    ''' 
    Get 2D Cosine Transform of Image
    '''
    return fftpack.dct(fftpack.dct(img.T, norm='ortho').T, norm='ortho')

def get_idct(coefficients):
    ''' 
    Get 2D Inverse Cosine Transform of Image
    '''
    return fftpack.idct(fftpack.idct(coefficients.T, norm='ortho').T, norm='ortho')

def apply_image_transformation(img, block_size, transform_function):
    '''
    Apply block-wise DCT or IDCT to an image.
    '''
    size = img.shape
    transformed_img = np.zeros(size, dtype=float)

    for y, x, macroblock in generate_macroblocks(img, block_size):
        transformed_img[y:y+block_size, x:x+block_size] = transform_function(macroblock)
        

    return transformed_img

def apply_quantization(transformed_image,block_size):

    size = transformed_image.shape
    quantized_image = np.zeros(size, dtype=float)
    
    for y, x, macroblock in generate_macroblocks(transformed_image, block_size):
        quantized_block = np.divide(macroblock, quantization_matrix)
        quantized_block = np.round(quantized_block)
        quantized_image[y:y+block_size, x:x+block_size] = quantized_block
    return quantized_image.astype(int)

def apply_dequantization(transformed_image,block_size):

    size = transformed_image.shape
    dequantized_image = np.zeros(size, dtype=float)

    for y, x, macroblock in generate_macroblocks(transformed_image, block_size):
        dequantized_block = np.multiply(macroblock, quantization_matrix)
        dequantized_image[y:y+block_size, x:x+block_size] = dequantized_block

    return dequantized_image


Compressao

In [ ]:
block_size = 8
img = ORIGINAL_IMAGE.astype(np.float64) - 128

residuals, options = process_image(img, block_size)
transformed_residuals = apply_image_transformation(residuals, block_size, get_dct)
quantized_residuals = apply_quantization(transformed_residuals,block_size)


Descompressao

In [ ]:
dequantized_residuals = apply_dequantization(quantized_residuals,block_size)
residuals_ = apply_image_transformation(dequantized_residuals, block_size, get_idct)
reconstructed_image = reconstruct_image(residuals_, block_size, options)
reconstructed_image = reconstructed_image.astype(np.float64) + 128


# Fazer a tranformada/fazer a transformada inversa - Fernando


In [14]:
def dct_2d_manual(matrix):
 
    N, M = matrix.shape
    dct_result = np.zeros((N, M))
    print(N,M)
    for k1 in range(N):
        for k2 in range(M):
            sum_value = 0
            for n1 in range(N):
                for n2 in range(M):
                    sum_value += matrix[n1, n2] * np.cos((np.pi/N)*(n1+0.5)*k1)*np.cos((np.pi/M)*(n2+0.5)*k2)

            dct_result[k1, k2] = sum_value
    
    return dct_result

In [ ]:
from scipy.fftpack import dct
usando_scipy=dct(ORIGINAL_IMAGE, type=2, norm='ortho')
plotImage(usando_scipy,'DCT SCIPY')

In [ ]:
dct_manual = dct_2d_manual(ORIGINAL_IMAGE)
plotImage(dct_manual,'DCT manual')

In [ ]:
def get_dct(img):
    ''' 
    Get 2D Cosine Transform of Image
    '''
    return fftpack.dct(fftpack.dct(img.T, norm='ortho').T, norm='ortho')

def get_idct(coefficients):
    ''' 
    Get 2D Inverse Cosine Transform of Image
    '''
    return fftpack.idct(fftpack.idct(coefficients.T, norm='ortho').T, norm='ortho')

def apply_image_transform(img, block_size, transform_function):
    '''
    Apply block-wise DCT or IDCT to an image.
    '''
    size = img.shape
    transformed_img = np.zeros(size, dtype=float)

    for y in range(0, size[0], block_size):
        for x in range(0, size[1], block_size):
            block = img[y:y+block_size, x:x+block_size]
            transformed_img[y:y+block_size, x:x+block_size] = transform_function(block)

    return transformed_img
    

## Inversa

In [ ]:
def idct_manual_2d(dct_matrix):
    N, M = dct_matrix.shape
    reconstructed_matrix = np.zeros((N, M))
    for n1 in range(N):
        for n2 in range(M):
            sum_value = 0
            for k1 in range(N):
                for k2 in range(M):
                    sum_value += dct_matrix[k1, k2] * \
                                 np.cos(np.pi * k1 * (2 * n1 + 1) / (2 * N)) * \
                                 np.cos(np.pi * k2 * (2 * n2 + 1) / (2 * M))
            reconstructed_matrix[n1, n2] = sum_value / (N * M)  # Normalização
    return reconstructed_matrix


In [ ]:
from scipy.fftpack import idct
inversa_usando_scipy=idct(usando_scipy, type=2, norm='ortho')
plotImage(inversa_usando_scipy,'IDCT SCIPY')

In [ ]:
inversa_manual=idct_manual_2d(dct_manual)
plotImage(inversa_manual,'DCT manual')


# Codificar/Decodidicar (huffman) - Araju


In [14]:
class Node:
    def __init__(self, freq, symbol=None, left=None, right=None):
        self.freq = freq
        self.symbol = symbol
        self.left = left
        self.right = right

    def __lt__(self, other):
        return self.freq < other.freq

def build_huffman_tree(frequencies):
    heap = [Node(freq, symbol) for symbol, freq in frequencies.items()]
    heapq.heapify(heap)

    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        merged = Node(left.freq + right.freq, left=left, right=right)
        heapq.heappush(heap, merged)

    return heap[0] if heap else None

def generate_huffman_codes(node, prefix="", codebook=None):
    if codebook is None:
        codebook = {}

    if node is not None:
        if node.symbol is not None:
            codebook[node.symbol] = prefix
        generate_huffman_codes(node.left, prefix + "0", codebook)
        generate_huffman_codes(node.right, prefix + "1", codebook)

    return codebook


# Salvar no arquivo a imagem comprimida/ descomprimir a imagem -  Araju


In [15]:
def encode_data(data, huffman_codes):
    return ''.join(huffman_codes[symbol] for symbol in data)

def decode_data(encoded_data, huffman_codes):
    reverse_huffman_codes = {v: k for k, v in huffman_codes.items()}
    current_code = ""
    decoded_data = []

    for bit in encoded_data:
        current_code += bit
        if current_code in reverse_huffman_codes:
            decoded_data.append(reverse_huffman_codes[current_code])
            current_code = ""

    return decoded_data


def save_to_file(original_image_shape, filename, huffman_codes, encoded_data):
    with open(filename, 'wb') as file:
        file.write(struct.pack('<I', original_image_shape[0]))
        file.write(struct.pack('<I', original_image_shape[1]))
        # Save the number of unique symbols
        file.write(struct.pack('<I', len(huffman_codes)))

        # Save the Huffman codes
        for symbol, code in huffman_codes.items():
            file.write(struct.pack('<i', symbol))  # Symbol as 4 bytes
            file.write(struct.pack('<B', len(code)))  # Length of the code
            file.write(code.encode())  # Code as bytes

        # Save the length of the encoded data in bits
        file.write(struct.pack('<I', len(encoded_data)))

        # Convert the encoded data to bytes
        buffer = 0
        buffer_length = 0
        byte_array = bytearray()

        for bit in encoded_data:
            buffer = (buffer << 1) | int(bit)
            buffer_length += 1

            if buffer_length == 8:
                byte_array.append(buffer)
                buffer = 0
                buffer_length = 0

        if buffer_length > 0:
            buffer <<= (8 - buffer_length)
            byte_array.append(buffer)
        print(sys.getsizeof(byte_array))
        file.write(byte_array)

def load_from_file(filename):
    with open(filename, 'rb') as file:
        # Read the shape of the original image
        original_image_shape = (struct.unpack('<I', file.read(4))[0], struct.unpack('<I', file.read(4))[0])
        # Read the number of unique symbols
        num_symbols = struct.unpack('<I', file.read(4))[0]

        # Read the Huffman codes
        huffman_codes = {}
        for _ in range(num_symbols):
            symbol = struct.unpack('<i', file.read(4))[0]
            code_length = struct.unpack('<B', file.read(1))[0]
            code = file.read(code_length).decode()
            huffman_codes[symbol] = code

        # Read the length of the encoded data in bits
        encoded_data_length = struct.unpack('<I', file.read(4))[0]

        # Read the encoded data as bits
        encoded_data = ''
        while len(encoded_data) < encoded_data_length:
            byte = file.read(1)
            if not byte:
                break
            byte_value = ord(byte)
            encoded_data += f'{byte_value:08b}'

        return original_image_shape, huffman_codes, encoded_data[:encoded_data_length]

def huffman_encode_matrix(matrix):
    flat_list = [item for sublist in matrix for item in sublist]
    frequencies = Counter(flat_list)
    huffman_tree = build_huffman_tree(frequencies)
    huffman_codes = generate_huffman_codes(huffman_tree)
    encoded_data = encode_data(flat_list, huffman_codes)
    return huffman_codes, encoded_data

def huffman_decode_matrix(encoded_data, huffman_codes, original_shape):
    decoded_flat_list = decode_data(encoded_data, huffman_codes)
    matrix = []
    index = 0
    for row_size in original_shape:
        matrix.append(decoded_flat_list[index:index + row_size])
        index += row_size
    return matrix



In [ ]:
# Random Matrix 8 x 8
matrix = [[1, 2, 3, 4, 5, 6, 7, 8],
          [2, 2, 2, 2, 2, 2, 2, 2],
          [3, 4, 5, 6, 7, 8, 9, 10],
          [4, 4, 4, 4, 4, 4, 4, 4],
          [5, 6, 7, 8, 9, 10, 11, 12],
          [6, 6, 6, 6, 6, 6, 6, 6],
          [7, 8, 9, 10, 11, 12, 13, 14],
          [8, 8, 8, 8, 8, 8, 8, 8]]
matrix = np.array(matrix)
original_shape = [len(row) for row in matrix]
huffman_codes, encoded_data = huffman_encode_matrix(matrix)
filename = 'matrix.huff'
save_to_file(matrix.shape, filename, huffman_codes, encoded_data)


loaded_orginal_image_shape, loaded_huffman_codes, loaded_encoded_data = load_from_file(filename)
decoded_matrix = huffman_decode_matrix(loaded_encoded_data, loaded_huffman_codes, original_shape)
print(matrix.shape == np.array(decoded_matrix).shape)
print(decoded_matrix == matrix)